In [ ]:
from dotenv import load_dotenv

load_dotenv('../.env')

## Teste de geração de perguntas

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_aws import BedrockEmbeddings, ChatBedrock
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.retrievers.multi_query import MultiQueryRetriever


llm = ChatBedrock(
    model_id='amazon.nova-micro-v1:0',
    temperature=.0,
    region='us-east-1',
    )

embeddings = BedrockEmbeddings(region_name='us-east-1')

from langchain_core.prompts import PromptTemplate

query = 'Em que situações posso ter minha matricula cancelada?'

prompt = """
Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar {k}
versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de um
banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar
o usuário a superar algumas das limitações da pesquisa de similaridade baseada em distância.
Forneça somente essas perguntas alternativas separadas por quebras de linha, sem textos adicionais.
Pergunta original: {questao}
"""
prompt_template = PromptTemplate.from_template(prompt)
prompt = prompt_template.format(questao=query, k=5)

response = llm.invoke(prompt)

In [ ]:
from pprint import pprint
pprint(response.content.split('\n'))

In [ ]:
from langchain_core.prompts import PromptTemplate

hyde_prompt = """
Você escreve artigos para o regulamento  dos cursos de graduação da UFRN.

Gere um melhor palpite de artigo hipotético que responde a pergunta: {pergunta}.

Siga cuidadosamente as instruções abaixo:
- Analise cuidadosamente a pergunta e as informações necessárias para responde-la.
- Formule um ou mais artigos abrangentes que respondem diretamente pergunta.
- Não adicione nenhum conhecimento externo.
- Use um tom formal, objetivo curto e conciso.
- Os artigos devem conter uma frase e deve responder somente a pergunta.
- Escreva somente as frases artigos, sem textos de agradecimento, de apresentação do resultado ou de explicações fora do contexto.
- Não escreva um número hipotetico dos artigos, somente frases.
- Não apresente o resultado com frases do tipo: "Aqui está o resultado".
- Cada artigo deve conter um contexto único e não deve ser igual aos outros

Alguns artigos de exemplo:

A oferta do componente curricular didático-pedagógico, sob a responsabilidadedosprogramas de Pós-Graduação, deve atender, de forma sistemática e articulada, às demandasdosprogramas de Pós-Graduação da UFRN.

É facultado aos discentes regularmente matriculados nos cursos de graduaçãodaUFRN,a matrícula em componentes curriculares isolados ofertados pelos Programas de Pós-GraduaçãodaUFRN, desde que autorizado pelo coordenador do curso de graduação e do coordenador doprogramade Pós-Graduação
"""
hyde_prompt_template = PromptTemplate.from_template(hyde_prompt)

In [ ]:
from langchain.schema import Document

def remove_duplicated_docs(documents):
    # Usar um conjunto para armazenar conteúdos únicos
    unique_documents = []
    seen_contents = set()

    for doc in documents:
        if doc.page_content not in seen_contents:
            unique_documents.append(doc)
            seen_contents.add(doc.page_content)

    return list(seen_contents)

remove_duplicated_docs(
    documents = [
    Document(page_content="Este é o conteúdo 1", metadata={"id": 1}),
    Document(page_content="Este é o conteúdo 2", metadata={"id": 2}),
    Document(page_content="Este é o conteúdo 1", metadata={"id": 3}),  # Duplicado
]
)

In [ ]:
from dotenv import load_dotenv

load_dotenv('../.env')

In [ ]:
import os

client = QdrantClient(
    location=os.environ["VECTOR_STORE_URL"],
    api_key=os.environ["VECTOR_STORE_API_KEY"]
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="regulamento-semantic",
    embedding=embeddings
)

In [ ]:
from datasets import Dataset

def pandas_to_ragas(df):
    '''
    Converts a Pandas DataFrame into a Ragas-compatible dataset
    
    Inputs:
        - df (Pandas DataFrame): The input DataFrame to be converted
        
    Returns:
        - ragas_testset (Hugging Face Dataset): A Hugging Face dataset compatible with the Ragas framework
    '''
    # Ensure all text columns are strings and handle NaN values
    text_columns = df.columns
    for col in text_columns:
        df[col] = df[col].fillna('').astype(str)
        
    # Convert 'contexts' to a list of lists
    df['reference_contexts'] = df['reference_contexts'].fillna('').astype(str).apply(eval)
    df['retrieved_contexts'] = df['retrieved_contexts'].fillna('').astype(str).apply(eval)
    
    # Converting the DataFrame to a dictionary
    data_dict = df.to_dict('list')
    
    # Loading the dictionary as a Hugging Face dataset
    ragas_testset = Dataset.from_dict(data_dict)
    
    return ragas_testset

In [ ]:
import time
import random

def invoke_llm_with_backoff(llm, prompt, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            return llm.invoke(prompt)
        except Exception as e:
            retries += 1
            wait_time = random.uniform(2 ** retries, 2 ** retries + 5)  # Exponential backoff
            print(f"Throttling error. Retrying in {wait_time:.2f} seconds...")
            time.sleep(wait_time)
    raise Exception("Max retries reached, could not invoke model.")

In [ ]:
import pandas as pd
from tqdm import tqdm
from ragas import evaluate
from ragas.metrics import NonLLMContextRecall, NonLLMContextPrecisionWithReference

ks = []
recalls =  []
# all_result_df = pd.DataFrame([])
models = [
    'amazon.nova-lite-v1:0',
    # 'amazon.nova-micro-v1:0',
    # 'meta.llama3-8b-instruct-v1:0',
    # 'mistral.mistral-7b-instruct-v0:2',
    # 'mistral.mixtral-8x7b-instruct-v0:1'
    ]

for model_name in models:
    for k in [20, 30]:

        df = pd.read_csv('../data/dataset_potiguana.csv')

        llm = ChatBedrock(
            model_id=model_name,
            temperature=.0,
            region='us-east-1',
            )

        retrieved_contexts = []
        ks = []

        for _, row in tqdm(df.iterrows()):
            query = row['user_input']

            # Generate queries
            prompt = prompt_template.format(questao=query, k=5)
            generated_queries = invoke_llm_with_backoff(llm=llm, prompt=prompt)
            queries_list = generated_queries.content.split('\n')

            # Aggregate queries
            query_contexts = []
            for q in queries_list:

                if q == '':
                    continue

                hyde_prompt = hyde_prompt_template.format(pergunta=q)
                hyde = invoke_llm_with_backoff(llm=llm, prompt=hyde_prompt)
                context_docs = vector_store.similarity_search(query=hyde.content, k=k)
                contexts = [c.page_content for c in context_docs]
                query_contexts += contexts

            contexts_cleaned = list(set(query_contexts))
            ks.append(len(contexts_cleaned))
            retrieved_contexts.append(contexts_cleaned)

        df['retrieved_contexts'] = pd.Series(retrieved_contexts)
        eval_dataset = pandas_to_ragas(df)

        metrics = [
            NonLLMContextPrecisionWithReference(threshold=0.95),
            NonLLMContextRecall(threshold=0.95)
            ]

        results = evaluate(dataset=eval_dataset, metrics=metrics)
        result_df = results.to_pandas()
        result_df['k'] = pd.Series(ks)
        result_df['model'] = model_name
        all_result_df = pd.concat([all_result_df, result_df], ignore_index=True)

all_result_df.to_csv('../multiple_query_hyde_eval_2.csv')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

all_result_df = pd.read_csv('../multiple_query_hyde_eval_2.csv') 

# Calcule a média de NonLLMContextRecall para cada valor de K
mean_recall = all_result_df.groupby('k')['non_llm_context_recall'].mean()
mean_precision = all_result_df.groupby('k')['non_llm_context_precision_with_reference'].mean()

plt.figure(figsize=(10, 6))
plt.plot(mean_recall.index, mean_recall.values, marker='o', linestyle='-', color='b', label='Mean Context Recall')
plt.plot(mean_precision.index, mean_precision.values, marker='s', linestyle='--', color='r', label='Mean Context Precision')
plt.title('Mean Context Recall and Precision vs. K', fontsize=16)
plt.xlabel('K', fontsize=14)
plt.ylabel('Mean Score', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)

# Define o intervalo de valores no eixo x de 10 em 10
plt.xticks(range(int(mean_recall.index.min()), int(mean_recall.index.max()) + 1, 10))  
plt.show()


In [ ]:
all_result_df.head(2)

In [ ]:
all_result_df.to_csv("../multiple_queries_in_k.csv")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Carregar os resultados consolidados
all_result_df = pd.read_csv("../multiple_queries_in_k.csv")
all_result_df = all_result_df[all_result_df['k'] <= 50]

# Calcule a média de NonLLMContextRecall para cada valor de K
mean_recall = all_result_df.groupby('k')['non_llm_context_recall'].mean()
mean_precision = all_result_df.groupby('k')['non_llm_context_precision_with_reference'].mean()

plt.figure(figsize=(12, 6))  # Aumentar tamanho da figura
plt.plot(mean_recall.index, mean_recall.values, marker='o', linestyle='-', color='b', label='Mean Context Recall')
# plt.plot(mean_precision.index, mean_precision.values, marker='s', linestyle='--', color='r', label='Mean Context Precision')
plt.title('Mean Context Recall and Precision vs. K', fontsize=16)
plt.xlabel('K', fontsize=14)
plt.ylabel('Mean Score', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)

# Mostrar apenas múltiplos de 5 no eixo X
plt.xticks(mean_recall.index[::5])  # Reduz o número de rótulos no eixo x
plt.show()

In [ ]:
import pandas as pd

# Carregar os resultados consolidados
all_result_df = pd.read_csv("../multiple_queries_in_k.csv")
all_result_df = all_result_df[all_result_df['k'] <= 50]

# Filtrar os valores específicos de k
selected_k_values = [5, 10, 15, 25, 30]
filtered_df = all_result_df[all_result_df['k'].isin(selected_k_values)]

# Calcular a média de NonLLMContextRecall e NonLLMContextPrecision para os valores filtrados
summary_df = filtered_df.groupby('k').agg({
    'non_llm_context_recall': 'mean',
    'non_llm_context_precision_with_reference': 'mean'
}).reset_index()

# Renomear as colunas para maior clareza
summary_df.rename(columns={
    'k': 'K',
    'non_llm_context_recall': 'Mean Recall',
    'non_llm_context_precision_with_reference': 'Mean Precision'
}, inplace=True)

# Exibir a tabela
print(summary_df)
